In [1]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [2]:
input_ = pd.read_excel('./file/input.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Url'].tolist()

总数量：2


['https://www.auveco.com/products/assortments/cabinetandcompartmentbinassortments',
 'https://www.auveco.com/products/bodyhardware/boltsbodyhardware/bodybolts']

In [3]:
a = 0

a

0

In [4]:
crawler_status = 'error'

crawler_status

'error'

In [5]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

url = input_.loc[a, 'Url']

dict_param = {}
list_param = url.split('?')[1].split('&') if '?' in url else []
for param in list_param:
    dict_param[param.split('=')[0]] = param.split('=')[1]

dict_param.pop('display', '')
dict_param['show'] = '1'
dict_param['page'] = '1'

url = url.split('?')[0] + '?'
for key, value in dict_param.items():
    url += f'{key}={value}&'

b = 0
while True:
    b += 1
    print(b)
    try:
        request_url = f'''https://www.auveco.com/api/cacheable/items?commercecategoryurl={url.split('?')[0].split('://www.auveco.com')[1]}&country=US&currency=USD&fieldset=search&include=facets&language=en&pricelevel=1&sort=commercecategory%3Adesc&use_pcv=T&limit=1&offset=0'''
        check_url = url[:-1]

        resp = requests.get(request_url,
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(10, 10))
        
        if resp.status_code == 200:
            dict_ = resp.json()
            
            if dict_['code'] == 200:
                break
    except KeyboardInterrupt:
        break
    except:
        continue

dict_

1


{'total': 12,
 'items': [{'pricelevel130': 349.17,
   'pricelevel251': 349.17,
   'pricelevel250': 349.17,
   'pricelevel255': 349.17,
   'pricelevel133': 349.17,
   'pricelevel254': 349.17,
   'pricelevel132': 349.17,
   'pricelevel253': 349.17,
   'pricelevel131': 349.17,
   'pricelevel252': 349.17,
   'pricelevel138': 349.17,
   'pricelevel259': 349.17,
   'pricelevel137': 349.17,
   'pricelevel258': 349.17,
   'pricelevel136': 349.17,
   'pricelevel257': 349.17,
   'pricelevel135': 349.17,
   'pricelevel256': 349.17,
   'ispurchasable_detail': {'inpersonalizedcatalog': True,
    'bystockavailability': False},
   'pricelevel139': 349.17,
   'pricelevel93': 349.17,
   'pricelevel92': 349.17,
   'pricelevel95': 349.17,
   'pricelevel94': 349.17,
   'pricelevel91': 349.17,
   'pricelevel90': 349.17,
   'pricelevel97': 349.17,
   'pricelevel96': 349.17,
   'pricelevel98': 349.17,
   'pricelevel141': 349.17,
   'pricelevel262': 349.17,
   'pricelevel140': 349.17,
   'pricelevel261': 349.

In [6]:
dict_['total'] <= 0

False

In [7]:
import math

df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                         'Url': url[:-1],
                         'SKU Count': dict_['total'],
                         'Page': math.ceil(dict_['total'] / 100)}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,No,Url,SKU Count,Page
0,1,https://www.auveco.com/products/assortments/ca...,12,1


In [8]:
crawler_status = 'ok'

crawler_status

'ok'

In [9]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
    df_temp.loc[0, 'Request_Url'] = request_url
    df_temp.loc[0, 'Check_Url'] = check_url
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [10]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel('./test_page.xlsx', index=False)

output_correct

,No,Url,SKU Count,Page
0,1,https://www.auveco.com/products/assortments/ca...,12,1


In [11]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[False],
                                            ignore_index=True)
    output_error.to_excel('./test_page_error.xlsx', index=False)

output_error

""
